In [174]:
import numpy as np

### Generate dataset

In [175]:
input_array = np.random.rand(100,2)

### Function to sort 2d array by column i

In [176]:
def merge_sort_2d_by_column(input_2d_array,by_column=0):
    accompany_index = 1 - by_column #index for the column that's not sort by but accompanying the sort
    sort_by_col=input_2d_array[:,by_column]
    acomp_by_col=input_2d_array[:,accompany_index]
    
    n = sort_by_col.shape[0]
    #print("input"+str(input_list))

    sorted_array = np.empty((0,2),float)
    if n>2: # If there are more than 2 pair of points in the array, split and deal with them recursively.
        array_1 = input_2d_array[0:n//2,:]
        array_2 = input_2d_array[n//2:,:]
        sorted_1 = merge_sort_2d_by_column(array_1,by_column)
        sorted_2 = merge_sort_2d_by_column(array_2,by_column)
        
        #Merge step
        i=0
        j=0
        for k in range(0,n):
            if (i <= (n//2) -1 ) & ( j <= n - (n//2) -1):#i and j are in range, other wise append the rest of the array
                if sorted_1[:,by_column][i] < sorted_2[:,by_column][j]:#when item i in sorted_1 is smaller
                    sorted_array = np.vstack((sorted_array,sorted_1[i,:]))
                    i += 1
                elif sorted_1[:,by_column][i] > sorted_2[:,by_column][j]:#when item j in sorted_2 is smaller
                    sorted_array = np.vstack((sorted_array,sorted_2[j,:]))
                    j += 1
            elif i>(n//2) -1:#When all coordinates in sorted_1 are consumed.
                sorted_array = np.vstack((sorted_array,sorted_2[j:,:]))
                break
            elif j>n - (n//2) -1:#When all coordinates in sorted_2 are consumed.
                sorted_array = np.vstack((sorted_array,sorted_1[i:,:]))
                break
    elif n == 2: # Base case, if there's only 2 pairs, sort them by compairing
        if input_2d_array[0,by_column]<=input_2d_array[1,by_column]:
            sorted_temp = np.vstack((input_2d_array[0,:],input_2d_array[1,:]))
            sorted_array = np.vstack((sorted_array,sorted_temp))
        else:
            sorted_temp = np.vstack((input_2d_array[1,:],input_2d_array[0,:]))
            sorted_array = np.vstack((sorted_array,sorted_temp))
    elif n == 1: # Base case, if there's only 1 pair of points, append them to sorted.
        sorted_array = np.vstack((sorted_array,input_2d_array[0,:]))
    #print("sorted="+str(sorted_list))
    return sorted_array

In [177]:
def euc_square_d(point_1:np.ndarray,point_2:np.ndarray):
    d = np.power((point_1[0] - point_2[0]),2) + np.power((point_1[1] - point_2[1]),2)
    return d    

In [178]:
def sort_by_y(px,py):
    Ly = np.empty((0,2),float)
    Ry = np.empty((0,2),float)
    n = px.shape[0]
    x_mid = (px[n//2-1,0] + px[n//2,0]) /2
    for i in range(0,py.shape[0]):
        if py[i,0] < x_mid:
            Ly = np.vstack((Ly,py[i,:]))
        else:
            Ry = np.vstack((Ry,py[i,:]))
    return Ly,Ry  

In [179]:
# Preprocessing, sort table by X and Y
px = merge_sort_2d_by_column(input_array,0)
py = merge_sort_2d_by_column(input_array,1)

In [180]:
#Lx Ly Rx Ry
#Lx: First half of px sorted by x
#Ly: First half of px sorted by y
#Rx: Second half of px sorted by x
#Ry: Second half of px sorted by y
#l1_l2 = closestpair(Lx,Ly)
#r1_r2 = closestpair(Rx,Ry)

def closest_pair(input_array):
    n = input_array.shape[0]
    min_pt = np.empty((0,2),float)
    if n == 2: #base case: 2pts, min is distance betwenn them
        min_val = euc_square_d(input_array[0,:],input_array[1,:])
        min_pt = np.vstack((input_array[0,:],input_array[1,:]))
        #print (str(n)+'case min='+str(min_val))  
        return min_val,min_pt
    
    elif n ==3: #base case: 3pts, min is distance betwenn any two of them
        min_val = euc_square_d(input_array[0,:],input_array[1,:])
        min_pt = np.vstack((input_array[0,:],input_array[1,:]))
        
        if euc_square_d(input_array[0,:],input_array[2,:]) < min_val:
            min_val = euc_square_d(input_array[0,:],input_array[2,:])
            min_pt = np.vstack((input_array[0,:],input_array[2,:]))  
            
        elif euc_square_d(input_array[1,:],input_array[2,:]) < min_val:
            min_val = euc_square_d(input_array[1,:],input_array[2,:])
            min_pt = np.vstack((input_array[1,:],input_array[2,:]))   
        #print (str(n)+'case min='+str(min_val)) 
        return min_val, min_pt
    
    elif n>3: # if more than 3 points, do recursive call
        
        px = merge_sort_2d_by_column(input_array,0)
        py = merge_sort_2d_by_column(input_array,1)
        LX = px[0:n//2,:]
        RX = px[n//2:,:]
        LY,RY = sort_by_y(px,py)
        #print(LX)
        l1,pair_l1 = closest_pair(LX)
        r1,pair_r1 = closest_pair(RX)

        if l1<r1:
            delta = np.power(l1,0.5)
            min_pt = pair_l1
        else:
            delta = np.power(r1,0.5)
            min_pt = pair_r1   

        # Closest split pair will be identified by the following
        x_bar = px[n//2-1,0]
        SY = np.empty((0,2),float)
        
        for i in range(0,n):
            if (py[i,0]> x_bar - delta) and (py[i,0]< x_bar + delta):
                SY = np.vstack((SY,py[i,:]))
        #print(n,i,SY,SY.shape[0])
        # brute force search closest pair in SY, it was proven that at most you need to search for 7 points ahead of p
        overall_min = np.power(delta,2)
        for i in range(0,SY.shape[0]-1):
            for j in range(i+1,min(7+i+1,SY.shape[0])):
                #print(i,j,SY)
                if euc_square_d(SY[i,:],SY[j,:]) < overall_min:
                    overall_min = euc_square_d(SY[i,:],SY[j,:])
                    min_pt = np.vstack((SY[i,:],SY[j,:]))   
        #print (str(n)+'case min='+str(overall_min)) 
        return overall_min, min_pt

In [181]:
closest_pair(input_array)

(9.54728320329424e-05, array([[0.81910391, 0.43311421],
        [0.80936112, 0.43385641]]))

In [182]:
def brute_search_cloest_pair(input_array):
    overall_min = euc_square_d(input_array[0,:],input_array[1,:])
    #print(overall_min)
    min_pt = np.empty((0,2),float)
    for i in range(0,input_array.shape[0]-1):
        for j in range(i+1,input_array.shape[0]):
            #print(i,j)
            if euc_square_d(input_array[i,:],input_array[j,:]) < overall_min:
                overall_min = euc_square_d(input_array[i,:],input_array[j,:])
                min_pt = np.vstack((input_array[i,:],input_array[j,:]))
    return overall_min,min_pt

In [183]:
brute_search_cloest_pair(merge_sort_2d_by_column(input_array,0))

(9.54728320329424e-05, array([[0.80936112, 0.43385641],
        [0.81910391, 0.43311421]]))

In [184]:
merge_sort_2d_by_column(input_array,0)

array([[1.84326207e-02, 5.87566058e-01],
       [4.91603063e-02, 6.53267536e-01],
       [5.65378486e-02, 9.37160068e-01],
       [5.78579408e-02, 9.93384641e-02],
       [9.01200202e-02, 7.44633980e-02],
       [9.15838805e-02, 3.92532153e-01],
       [9.38881011e-02, 4.27111389e-02],
       [9.51196810e-02, 4.62453205e-01],
       [1.02251242e-01, 3.80049117e-01],
       [1.11744565e-01, 2.09262697e-02],
       [1.22562923e-01, 5.59539148e-01],
       [1.25626384e-01, 2.27300275e-01],
       [1.38176126e-01, 3.17053738e-01],
       [1.53594955e-01, 4.39256869e-01],
       [1.57041280e-01, 7.22704440e-01],
       [1.67765668e-01, 1.65512996e-01],
       [1.72282519e-01, 1.19447691e-01],
       [1.78030903e-01, 8.27550396e-01],
       [1.87936036e-01, 3.21042405e-01],
       [1.96226649e-01, 3.79516565e-01],
       [1.96661742e-01, 8.90923680e-01],
       [2.06844849e-01, 9.07953229e-01],
       [2.11512097e-01, 6.94119020e-01],
       [2.18047108e-01, 5.44795284e-01],
       [2.255376

In [185]:
euc_square_d(merge_sort_2d_by_column(input_array,0)[3,:],merge_sort_2d_by_column(input_array,0)[6,:])

0.004504826402976966